In [1]:
!pip install transformers

In [2]:
!pip install pandas

In [3]:
!pip install scikit-learn


In [4]:
from transformers import BertModel, BertTokenizerFast
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from transformers import BertModel, BertTokenizerFast
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.multiclass import unique_labels
import pandas as pd
import numpy as np
import os

/home/yagudcor/.conda/envs/coral_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Correct file path
data_path = "/home/yagudcor/translated_distinct_records_final_version.csv"

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Dataset not found at {data_path}. Ensure it's uploaded and linked to your kernel.")

# Load the dataset
data = pd.read_csv(data_path)
data

,id,transcript,intent,slots:,cam,transcript_hebrew
0,9024,event,calendar_set,[],calendar_set(),אני צריך שתקבעי לי אירוע
1,7874,i need an event three days from now scheduled ...,calendar_set,['date=three days from now' 'person=amy'],"calendar_set(date=three days from now, person=...",אני צריך שתקבעי לי אירוע מתוכנן בעוד שלושה ימי...
2,5822,turn up,audio_volume_up,[],audio_volume_up(),אני צריך שתגבירי את הווליום
3,5013,brighten the lights a little bit,iot_hue_lightup,[],iot_hue_lightup(),אני צריכה שתוריד קצת את הבהירות
4,3033,what's the current weather,weather_query,[],weather_query(),אני רוצה לדעת מה מזג האוויר הנוכחי
...,...,...,...,...,...,...
11505,695,call the restaurant to see if the delivery is ...,takeaway_query,['business_type=restaurant' 'order_type=delive...,"takeaway_query(business_type=restaurant, order...",התקשר למסעדה כדי לראות אם המשלוח בדרך
11506,2823,what day does easter fall on this year,datetime_query,['event_name=easter' 'date=this year'],"datetime_query(event_name=easter, date=this year)",באיזה יום חל חג הפסחא השנה
11507,17173,check someone's recent emails,email_query,[],email_query(),תבדוק מיילים אחרונים של מישהו
11508,9845,resuming playing my current audio book,play_audiobook,['player_setting=resuming' 'media_type=audio b...,"play_audiobook(player_setting=resuming, media_...",תחזור לנגן את הספר הקולי שלי


In [6]:
le = LabelEncoder()
data['intent_num'] = le.fit_transform(data['intent'])
map_intent = dict(zip(le.transform(le.classes_), le.classes_))
map_intent

{0: 'addcontact',
 1: 'alarm_query',
 2: 'alarm_remove',
 3: 'alarm_set',
 4: 'audio_volume_down',
 5: 'audio_volume_mute',
 6: 'audio_volume_other',
 7: 'audio_volume_up',
 8: 'calendar_query',
 9: 'calendar_remove',
 10: 'calendar_set',
 11: 'cleaning',
 12: 'coffee',
 13: 'convert',
 14: 'cooking_query',
 15: 'cooking_recipe',
 16: 'createoradd',
 17: 'currency',
 18: 'datetime_convert',
 19: 'datetime_query',
 20: 'definition',
 21: 'email_addcontact',
 22: 'email_query',
 23: 'email_querycontact',
 24: 'email_sendemail',
 25: 'events',
 26: 'factoid',
 27: 'game',
 28: 'general_greet',
 29: 'general_joke',
 30: 'general_quirky',
 31: 'greet',
 32: 'hue_lightdim',
 33: 'hue_lightoff',
 34: 'hue_lightup',
 35: 'iot_cleaning',
 36: 'iot_coffee',
 37: 'iot_hue_lightchange',
 38: 'iot_hue_lightdim',
 39: 'iot_hue_lightoff',
 40: 'iot_hue_lighton',
 41: 'iot_hue_lightup',
 42: 'iot_wemo_off',
 43: 'iot_wemo_on',
 44: 'joke',
 45: 'lists_createoradd',
 46: 'lists_query',
 47: 'lists_remo

In [7]:
# Apply filter (update the condition based on your needs)
# Example: Filter rows where the 'intent' column matches Siri intents
filter_condition = data['intent'].isin([
    'calendar_set', 'calendar_query', 'weather_query', 'play_music', 'play_audiobook',
    'play_radio', 'email_query', 'sendemail', 'alarm_query', 'alarm_set',
    'datetime_query', 'social_post', 'qa_factoid', 'general_query', 'news_query'
])
filtered_data = data[filter_condition]

# Stop execution if no rows match the filter
if filtered_data.empty:
    raise ValueError("Filtered dataset is empty. No matching rows found with the given condition.")

# Display filtered data (optional)
print(filtered_data.head())
print(len(filtered_data))

      id                                         transcript         intent  \
0   9024                                              event   calendar_set   
1   7874  i need an event three days from now scheduled ...   calendar_set   
4   3033                         what's the current weather  weather_query   
8  16742  check my email for new emails during the last ...    email_query   
9   5172               please wake me up at seven thirty am      alarm_set   

                                      slots:  \
0                                         []   
1  ['date=three days from now' 'person=amy']   
4                                         []   
8                         ['time=last hour']   
9                   ['time=seven thirty am']   

                                                 cam  \
0                                     calendar_set()   
1  calendar_set(date=three days from now, person=...   
4                                    weather_query()   
8                 

In [13]:
input = list(filtered_data['transcript_hebrew'])
print(input)
print(len(input))

['אני צריך שתקבעי לי אירוע', 'אני צריך שתקבעי לי אירוע מתוכנן בעוד שלושה ימים עם איימי', 'אני רוצה לדעת מה מזג האוויר הנוכחי', 'עיין בדוא"ל שלי להודעות דוא"ל חדשות במהלך השעה האחרונה', 'תשימי לי שעון מעורר לשבע וחצי בבוקר', 'את יכולה להראות לי את השעונים המתוכננים?', 'אני רוצה שתזכירי לי את הפגישה הבאה עם חברה שלי בשעה שמונה אחר הצהריים ביום ראשון הבא', 'אני רוצה שתנגני את רשימת ההשמעה טקטיקות מאפליקציית מוזיקה', 'תבדקי לי מה מזג האוויר', "את יכולה לבדוק מתי אני אמור להיפגש פעם הבאה עם ג'ו?", 'אנא הזכירי לי על פגישת עסקים באחת עשרה בבוקר', 'אנא תציג לי את השעון המעורר שקבעת לפגישה של יום שלישי', 'האם אני צריך ללבוש בגדים חמים היום?', 'תאונת רכבת', 'האם יש לי תזכורות צפויות', 'פייסבוק משיקה תג שנוי במחלוקת כדי לפצח חדשות מזויפות', 'הורד אוצר מילים מסוים', 'האם תוכל בבקשה ליצור אירוע חדש בלוח השנה שלי', 'תראה לי את כל האימיילים האחרונים מאנה', 'הפעל את ספר השמע', 'ספר לי את הפגישות שלי ביום רביעי בבוקר', 'קבע שעון מעורר ליום חמישי בשעה שבע אחר הצהריים', 'הוסף פגישה לרופא שיניים ליום שישי

In [9]:
# # Import necessary libraries
# from transformers import AutoTokenizer, AutoModel

# # Load the tokenizer and model for AlephBERT-Gimmel
# tokenizer = AutoTokenizer.from_pretrained("dicta-il/alephbertgimmel-base")
# model = AutoModel.from_pretrained("dicta-il/alephbertgimmel-base")

# # model_name = 'onlplab/alephbert-base'
# # model = BertModel.from_pretrained('onlplab/alephbert-base')
# # tokenizer = BertTokenizerFast.from_pretrained('onlplab/alephbert-base')
# #Tokenize and pad the texts
# encoded_inputs = tokenizer(input[:1000], padding=True,
#                             truncation=True,
#                             return_tensors='pt')


In [14]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dicta-il/alephbertgimmel-base")
model = AutoModel.from_pretrained("dicta-il/alephbertgimmel-base")

# Your input data
input_data = list(filtered_data['transcript_hebrew'])  # Replace with your actual input

# Define batch size
batch_size = 1000

# Store embeddings
sentence_embeddings = []

# Process data in batches
for i in range(0, len(input_data), batch_size):
    # Select a batch of data
    batch = input_data[i:i + batch_size]
    
    # Tokenize the batch
    encoded_inputs = tokenizer(batch, padding=True, truncation=True, max_length=512, return_tensors='pt')
    
    # Pass the batch through the model
    with torch.no_grad():
        outputs = model(**encoded_inputs)
    
    # Extract token embeddings
    token_embeddings = outputs.last_hidden_state  # Shape: [batch_size, seq_len, hidden_dim]
    
    # Use mean pooling to get a fixed-size embedding for each sentence
    batch_embeddings = token_embeddings.mean(dim=1).numpy()  # Mean pooling along the seq_len dimension
    sentence_embeddings.extend(batch_embeddings)

# Convert sentence_embeddings to a NumPy array
sentence_embeddings = np.array(sentence_embeddings)

# Verify the shape of embeddings
print("Embeddings shape:", sentence_embeddings.shape)

Some weights of BertModel were not initialized from the model checkpoint at dicta-il/alephbertgimmel-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embeddings shape: (5356, 768)


In [15]:
# Prepare labels
labels = list(filtered_data['intent_num'])[:len(sentence_embeddings)]

# Ensure the shapes of embeddings and labels match
assert len(sentence_embeddings) == len(labels), "Mismatch between embeddings and labels!"

In [16]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings, labels, test_size=0.2, random_state=42)

# Print results
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (4284, 768)
Test set shape: (1072, 768)


In [10]:
# with torch.no_grad():
#   outputs = model(**encoded_inputs)

In [11]:
# # the embedding of the first sentnce with alephbert
# outputs[0]

tensor([[[ 1.6602,  0.5846, -0.0294,  ...,  0.4390,  0.3201,  0.0481],
         [-0.5163, -1.6205,  0.5685,  ..., -0.1674,  1.1498,  0.4393],
         [-1.5847, -0.7396,  0.7437,  ...,  0.1873,  0.3119, -0.1438],
         ...,
         [-0.2656,  0.2142,  0.0436,  ...,  1.5440, -0.2401,  0.0954],
         [-0.2697,  0.2564,  0.0662,  ...,  1.5109, -0.2354,  0.0935],
         [-0.2840,  0.2542,  0.0440,  ...,  1.5099, -0.2542,  0.0886]],

        [[ 1.5048,  0.8769, -0.1702,  ...,  0.2410,  0.2754, -0.0217],
         [-0.2341, -1.4271,  0.6103,  ..., -0.2670,  0.9646,  0.4026],
         [-1.2571, -0.8067,  0.5589,  ...,  0.3366,  0.2412, -0.1412],
         ...,
         [-0.2228,  0.2258,  0.1108,  ...,  1.3948, -0.2110,  0.0819],
         [-0.2442,  0.2403,  0.0927,  ...,  1.4194, -0.2490,  0.0786],
         [-0.2247,  0.2175,  0.1252,  ...,  1.3698, -0.1818,  0.0889]],

        [[ 1.6711,  0.9177,  0.1590,  ...,  0.7145,  0.2621,  0.1885],
         [ 0.0871, -1.0955,  1.0306,  ...,  0

In [13]:
# token_embeddings = outputs.last_hidden_state

In [18]:
# sentence_embeddings = []
# for embedding in token_embeddings:
#   sentence_embeddings.append(embedding.flatten().numpy())

In [19]:
# sentence_embeddings = np.array(sentence_embeddings)

In [22]:
# labels = list(filtered_data['intent_num'])[:1000]

In [23]:
# X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings, labels, test_size=0.2, random_state=42)

1 - RandomForestClassifier

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


unique_labels_in_test = np.unique(y_test)

print(classification_report(
    y_test,
    y_pred,
    labels=unique_labels_in_test,
    target_names=[le.classes_[i] for i in unique_labels_in_test]
))


Accuracy: 81.72%
                precision    recall  f1-score   support

   alarm_query       0.93      0.52      0.67        27
     alarm_set       0.91      0.73      0.81        41
calendar_query       0.79      0.71      0.75       118
  calendar_set       0.68      0.96      0.80       156
datetime_query       0.98      0.79      0.87        62
   email_query       0.92      0.86      0.89        88
    news_query       0.79      0.85      0.82        96
play_audiobook       1.00      0.24      0.39        37
    play_music       0.74      0.93      0.82       129
    play_radio       0.93      0.50      0.65        56
    qa_factoid       0.80      0.90      0.85        98
   social_post       0.91      0.79      0.85        52
 weather_query       1.00      0.95      0.97       112

      accuracy                           0.82      1072
     macro avg       0.88      0.75      0.78      1072
  weighted avg       0.84      0.82      0.81      1072



2 - logistic regression

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

lr_clf = LogisticRegression(max_iter=1000, random_state=42)
lr_clf.fit(X_train, y_train)


y_pred_lr = lr_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_lr)
print(f"Accuracy: {accuracy * 100:.2f}%")


unique_labels_in_test = np.unique(y_test)
target_names = [le.classes_[i] for i in unique_labels_in_test]


print(classification_report(
    y_test,
    y_pred_lr,
    labels=unique_labels_in_test,
    target_names=target_names
))


Accuracy: 89.55%
                precision    recall  f1-score   support

   alarm_query       0.96      0.85      0.90        27
     alarm_set       0.83      0.93      0.87        41
calendar_query       0.85      0.82      0.84       118
  calendar_set       0.87      0.92      0.89       156
datetime_query       0.98      0.87      0.92        62
   email_query       0.96      0.97      0.96        88
    news_query       0.92      0.90      0.91        96
play_audiobook       0.84      0.70      0.76        37
    play_music       0.89      0.90      0.89       129
    play_radio       0.86      0.77      0.81        56
    qa_factoid       0.83      0.95      0.89        98
   social_post       0.94      0.92      0.93        52
 weather_query       0.97      0.96      0.97       112

      accuracy                           0.90      1072
     macro avg       0.90      0.88      0.89      1072
  weighted avg       0.90      0.90      0.90      1072



/home/yagudcor/.conda/envs/coral_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


3 -- SVM

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


svm_clf = SVC(kernel='linear', random_state=42)  # Use 'linear' kernel for high-dimensional data
svm_clf.fit(X_train, y_train)


y_pred_svm = svm_clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred_svm)
print(f"Accuracy: {accuracy * 100:.2f}%")


unique_labels_in_test = np.unique(y_test)
target_names = [le.classes_[i] for i in unique_labels_in_test]


print(classification_report(
    y_test,
    y_pred_svm,
    labels=unique_labels_in_test,
    target_names=target_names
))


Accuracy: 88.15%
                precision    recall  f1-score   support

   alarm_query       0.92      0.89      0.91        27
     alarm_set       0.84      0.93      0.88        41
calendar_query       0.85      0.79      0.82       118
  calendar_set       0.82      0.93      0.87       156
datetime_query       1.00      0.89      0.94        62
   email_query       0.98      0.94      0.96        88
    news_query       0.92      0.89      0.90        96
play_audiobook       0.77      0.65      0.71        37
    play_music       0.85      0.88      0.87       129
    play_radio       0.84      0.73      0.78        56
    qa_factoid       0.83      0.92      0.87        98
   social_post       0.94      0.92      0.93        52
 weather_query       0.95      0.94      0.95       112

      accuracy                           0.88      1072
     macro avg       0.89      0.87      0.88      1072
  weighted avg       0.88      0.88      0.88      1072



transfer learning with fine-tuning using a pre-trained BERT model (onlplab/alephbert-base)

with early stop

In [20]:
from transformers import BertModel, BertTokenizerFast, AdamW
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.multiclass import unique_labels
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")



le = LabelEncoder()
data['intent_num'] = le.fit_transform(data['intent'])
map_intent = dict(zip(le.transform(le.classes_), le.classes_))


intent_counts = data['intent_num'].value_counts()
print("Class distribution:")
print(intent_counts)

input_texts = list(data['transcript_hebrew'])
model_name = 'onlplab/alephbert-base'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)


encoded_inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt')
labels = torch.tensor(data['intent_num'].values)


train_size = 0.8
num_samples = len(labels)
indices = np.arange(num_samples)
np.random.shuffle(indices)
train_indices = indices[:int(train_size * num_samples)]
test_indices = indices[int(train_size * num_samples):]

train_inputs = {key: val[train_indices] for key, val in encoded_inputs.items()}
test_inputs = {key: val[test_indices] for key, val in encoded_inputs.items()}
train_labels = labels[train_indices]
test_labels = labels[test_indices]

class BertForIntentClassification(torch.nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertForIntentClassification, self).__init__()
        self.bert = bert_model
        self.classifier = torch.nn.Linear(bert_model.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # CLS token
        logits = self.classifier(cls_output)
        return logits

num_classes = len(map_intent)
fine_tuned_model = BertForIntentClassification(model, num_classes)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(fine_tuned_model.parameters(), lr=2e-5)

epochs = 5
batch_size = 16

train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


Class distribution:
intent_num
10    803
56    620
88    559
8     558
30    544
     ... 
25      1
13      1
11      1
76      1
87      1
Name: count, Length: 91, dtype: int64


Some weights of BertModel were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
# Training loop with Early Stopping
fine_tuned_model.train()

best_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(epochs):
    total_loss = 0

    # Training
    fine_tuned_model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()
        outputs = fine_tuned_model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_loss:.4f}")

    # Validation
    fine_tuned_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = fine_tuned_model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(test_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        early_stop_counter = 0
        # Optionally save the best model
        torch.save(fine_tuned_model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        print(f"No improvement for {early_stop_counter} epochs. Patience left: {patience - early_stop_counter}")

    if early_stop_counter >= patience:
        print("Early stopping triggered. Training stopped.")
        break

# Load the best model if saved
fine_tuned_model.load_state_dict(torch.load("best_model.pth"))

# Evaluation loop
fine_tuned_model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = fine_tuned_model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

present_classes = unique_labels(true_labels, predictions)
present_class_names = [map_intent[i] for i in present_classes]

print("Classification Report:")
print(classification_report(true_labels, predictions, target_names=present_class_names))


Epoch 1/5, Training Loss: 1.4361
Epoch 1/5, Validation Loss: 0.7884
Epoch 2/5, Training Loss: 0.5256
Epoch 2/5, Validation Loss: 0.7274
Epoch 3/5, Training Loss: 0.2985
Epoch 3/5, Validation Loss: 0.7552
No improvement for 1 epochs. Patience left: 4
Epoch 4/5, Training Loss: 0.1803
Epoch 4/5, Validation Loss: 0.7825
No improvement for 2 epochs. Patience left: 3
Epoch 5/5, Training Loss: 0.1385
Epoch 5/5, Validation Loss: 0.8355
No improvement for 3 epochs. Patience left: 2
Test Accuracy: 82.75%
Classification Report:
                          precision    recall  f1-score   support

             alarm_query       0.81      0.96      0.88        26
            alarm_remove       0.71      0.56      0.63         9
               alarm_set       0.97      0.79      0.87        43
       audio_volume_down       0.60      0.60      0.60        10
       audio_volume_mute       0.78      0.82      0.80        22
      audio_volume_other       1.00      0.40      0.57         5
         audio

increase num_epocs

In [22]:
epochs = 30
batch_size = 16

train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [23]:
# Training loop with Early Stopping
fine_tuned_model.train()

best_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(epochs):
    total_loss = 0

    # Training
    fine_tuned_model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()
        outputs = fine_tuned_model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_loss:.4f}")

    # Validation
    fine_tuned_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = fine_tuned_model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(test_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        early_stop_counter = 0
        # Optionally save the best model
        torch.save(fine_tuned_model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        print(f"No improvement for {early_stop_counter} epochs. Patience left: {patience - early_stop_counter}")

    if early_stop_counter >= patience:
        print("Early stopping triggered. Training stopped.")
        break

# Load the best model if saved
fine_tuned_model.load_state_dict(torch.load("best_model.pth"))

# Evaluation loop
fine_tuned_model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = fine_tuned_model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

present_classes = unique_labels(true_labels, predictions)
present_class_names = [map_intent[i] for i in present_classes]

print("Classification Report:")
print(classification_report(true_labels, predictions, target_names=present_class_names))


Epoch 1/30, Training Loss: 0.3306
Epoch 1/30, Validation Loss: 0.7451
Epoch 2/30, Training Loss: 0.1951
Epoch 2/30, Validation Loss: 0.8041
No improvement for 1 epochs. Patience left: 4
Epoch 3/30, Training Loss: 0.1205
Epoch 3/30, Validation Loss: 0.8442
No improvement for 2 epochs. Patience left: 3
Epoch 4/30, Training Loss: 0.0999
Epoch 4/30, Validation Loss: 0.8407
No improvement for 3 epochs. Patience left: 2
Epoch 5/30, Training Loss: 0.0810
Epoch 5/30, Validation Loss: 0.8790
No improvement for 4 epochs. Patience left: 1
Epoch 6/30, Training Loss: 0.0621
Epoch 6/30, Validation Loss: 0.9323
No improvement for 5 epochs. Patience left: 0
Early stopping triggered. Training stopped.
Test Accuracy: 82.93%
Classification Report:
                          precision    recall  f1-score   support

             alarm_query       0.83      0.96      0.89        26
            alarm_remove       0.83      0.56      0.67         9
               alarm_set       0.94      0.77      0.85       